In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt

from astropy.io import fits
from matplotlib.colors import LogNorm, TwoSlopeNorm
import numpy as np
import scipy.io
import hcipy

import pastis.util as util
from pastis.config import CONFIG_PASTIS
from pastis.simulators.luvoir_imaging import LuvoirA_APLC
from pastis.simulators.scda_telescopes import HexRingAPLC

os.chdir('../ULTRA')
from config import CONFIG_ULTRA

In [ ]:
data_path = CONFIG_ULTRA.get('local_path', 'local_data_path')
analysis_path = CONFIG_ULTRA.get('local_path', 'local_analysis_path')

input_misalignments = np.genfromtxt(os.path.join(data_path,'INPUT_MISALIGNMENTS.txt'),  delimiter=';')
output_misalignments = np.genfromtxt(os.path.join(data_path,'RESIDUAL_MISALIGNMENTS.txt'),  delimiter=';')

In [ ]:
optics_dir = os.path.join(util.find_repo_location(), 'data', 'SCDA')
NUM_RINGS = 2
sampling = 4

coron_psf = []
for wv in range(475, 550, 25):
    tel2 = HexRingAPLC(optics_dir, NUM_RINGS, sampling, wv)

    unaberrated_psf, ref, intermediates = tel2.calc_psf(ref=True, display_intermediate=False, 
                                      return_intermediate='intensity',
                                      norm_one_photon=True)

    norm = np.max(ref)
    normalized_unaberrated_psf = unaberrated_psf / norm
    coron_psf.append(normalized_unaberrated_psf)
    unaberr_roi = normalized_unaberrated_psf * tel2.dh_mask
    contrast_floor = np.mean(unaberr_roi[np.where(tel2.dh_mask != 0)])
    print("contrast_floor:", contrast_floor, "wavelength:", tel2.wvln)

In [ ]:
scaled_coron_psf = []
for i in range(0, 3):
    copy_psf = coron_psf[i].copy()
    copy_psf.grid = copy_psf.grid.scaled(1/(500 * 1e-9 / tel2.diam))
    scaled_coron_psf.append(copy_psf)


In [ ]:
#broadband = scaled_coron_psf[0] + scaled_coron_psf[1] + scaled_coron_psf[2]
broadband = (coron_psf[0] + coron_psf[1] + coron_psf[2])/3

In [ ]:
plt.figure(figsize = (10, 5))
plt.subplot(1, 2, 1)
hcipy.imshow_field(np.log10(coron_psf[0]), cmap='inferno')
plt.colorbar()

plt.subplot(1, 2, 2)
hcipy.imshow_field(np.log10(broadband), cmap='inferno')
plt.colorbar()

In [ ]:
copy_psf = normalized_unaberrated_psf.copy()
copy_psf.grid = copy_psf.grid.scaled(1/(500 * 1e-9 / tel2.diam))

hcipy.imshow_field(np.log10(copy_psf), cmap='inferno')

hcipy.write_fits(copy_psf, os.path.join(analysis_path, f'int_coron_{tel2.wvln *1e9 :.0f}.fits'))

In [ ]:
# copy_psf = normalized_unaberrated_psf.copy()
# copy_psf.grid = copy_psf.grid.scaled(1/(700 * 1e-9 / tel2.diam))

# plt.figure(figsize = (12, 5))
# r1, profile1, _, _ = hcipy.radial_profile(copy_psf, 0.15, statistic = 'mean')
# plt.plot(r1, np.log10(profile1))
# plt.show()

In [ ]:
mono_contrast = {"475": 4.1708498716575394e-11, 
                       "480": 4.1715715775570424e-11,
                       "485": 4.168976012133777e-11, 
                       "490": 4.168787270144789e-11,
                       "495": 4.167001672197429e-11,
                       "500": 4.171337358217274e-11, 
                       "505": 4.1687872701454613e-11, 
                       "510": 4.169515132321805e-11,
                       "515": 4.1652345639236236e-11,
                       "520": 4.1711895834213314e-11,
                       "525": 4.1715715775571277e-11,
                       "530": 4.171138806756271e-11,
                       "535": 4.1674416722877654e-11,
                       "540": 4.161060449198776e-11
                      }

In [ ]:
c_list = []
for wvln in range(475, 530, 5):
    print(wvln)
    contrast = mono_contrast[str(wvln)]
    c_list.append(contrast)

    
print("Broadband contrast:", np.mean(np.array(c_list)))

In [ ]:
optics_input = os.path.join(util.find_repo_location(), CONFIG_PASTIS.get('LUVOIR', 'optics_path_in_repo'))
sampling = CONFIG_PASTIS.getfloat('LUVOIR', 'sampling')

luvoir = LuvoirA_APLC(optics_input, "small", sampling)

unaberrated_coro_psf, ref = luvoir.calc_psf(ref=True, display_intermediate=False, norm_one_photon=True)
norm = np.max(ref)
dh_intensity = (unaberrated_coro_psf / norm) * luvoir.dh_mask
contrast_floor = np.mean(dh_intensity[np.where(luvoir.dh_mask != 0)])
print(f'static contrast floor for luvoirA small APLC design: {contrast_floor}, wavlength:{luvoir.wvln}')

In [ ]:
int_coron_500 =  hcipy.read_fits(os.path.join(analysis_path, 'int_coron_500.fits'))
int_coron_475 =  hcipy.read_fits(os.path.join(analysis_path, 'int_coron_475.fits'))
int_coron_525 =  hcipy.read_fits(os.path.join(analysis_path, 'int_coron_525.fits'))

int_broadband = int_coron_500 + int_coron_475 + int_coron_525

In [ ]:
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(np.log10(int_coron_500), cmap='inferno')
plt.colorbar()

plt.subplot(1, 2, 2)
plt.imshow(np.log10(int_broadband), cmap='inferno')
plt.colorbar()